### 6.1.1 Bandit算法简介
#### 一. 更改推荐流程
1. 现在,我们更改推荐流程:  
 1. 首先, 从"相似内容推荐", "相似好友推荐", "热门推荐"中选择一种推荐策略  
 2. 然后再使用选择的种策略取推荐物品.  
选择推荐方式的算法, 就是Bandit

#### 二. Bandit算法家族
1. Bandit算法家族解决什么问题  
 推荐所发的EE问题: 即`Exploit－Explore`问题
 
2. `累计遗憾`衡量选择的好坏  
 Bandit定义选择的好坏, 是看这个选择带来的遗憾有多少. 遗憾越少, 选择越好  
 $${ R }_{ T }=\sum _{ t=1 }^{ T }{ \left( { w }_{ opt }-{ w }_{ B\left( i \right)  } \right)  } \\ \quad =T*{ w }^{ * }-\sum _{ t=1 }^{ T }{ { w }_{ B\left( i \right)  } } $$  
  1. ${ R }_{ T }$ : $T$次选择带来的遗憾  
  2. ${ w }_{ opt }$ : 最优选择的价值(wealth,optional)  
  3. ${ w }_{ B\left( i \right)  }$ : $B\left( i \right)$次选择的价值
  
3. 多臂老虎机问题  
 多臂老虎机, 就是从多台老虎机中, 选择其中某一个老虎机进行赌博. 这样的操作重复多次, 使多次的累积收益最大. 多臂的选择问题, 就是Bandit算法. 放在推荐系统中, 就是 :  
 1. 臂 : 每次的候选项. 可以是具体物品, 也可以是推荐策略, 也可能是物品类别  
 2. 回报 : 用户是否对推荐结果喜欢.喜欢就是正回报, 未作出反应就是负回报或0回报  
 3. 环境 : 推荐系统面临的用户
 
4. 下面, 我们依次介绍几个Bandit算法

### 6.1.2 汤普森采样算法

#### 一. 汤普森采样的思路
1. 如何选择哪个推荐策略 :  
 假设每个臂可以产生收益的概率为$p$. 然后从每个臂的概率分布$p$中, 抽取一个随机数, 按照这个随机数排序, 输出产生随机数最大的那个臂推荐的物品
2. 臂的概率分布$p$ :  
 p的分布服从$Beta$分布, 即$$f\left( p \right) =\frac { { p }^{ \alpha -1 }{ \left( 1-p \right)  }^{ \beta -1 } }{ \int _{ 0 }^{ 1 }{ { u }^{ \alpha -1 }{ \left( 1-u \right)  }^{ \beta -1 } }  } $$  
3. 如下使用`pymc3`生成Beta分布的值, 再用numpy找到最大值的臂

In [1]:
import pymc3
import numpy as np

/devkits/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with pymc3.Model() as coin_model: 
    a = pymc3.Beta('Betas_a',alpha=95,beta=1,shape=1)
    b = pymc3.Beta('Betas_b',alpha=9,beta=91,shape=1)
    c = pymc3.Beta('Betas_c',alpha=96,beta=1,shape=1)
    d = pymc3.Beta('Betas_d',alpha=9,beta=4,shape=1)
    var_list = [a.random(),b.random(),c.random(),d.random()]
    print(var_list)
    print(np.argmax(var_list))

[array(0.98268633), array(0.09197584), array(0.99876265), array(0.7574932)]
2


### 6.1.3 UCB算法

#### 一. 何为UCB算法
1. UCB算法, 全称$upper confidence bound$, 即置信区间上界  
     UCB为每个候选臂评分, 然后选择评分最高的那个臂. 而且每次输出后观察用户反馈更新候选臂的参数
2. 如果说汤普森采样算法, 需要对每个臂给出一个先验的$\alpha$和$\beta$, 这引入了不确定性. 那么UCB算法, 则是一种确定的, 完全通过`count`次数实现的算法.UCB算法对每个臂的评分方法是 : $${ \bar { { x }_{ t } }  }\left( t \right) +\sqrt { \frac { 2\ln { t }  }{ { T }_{ j,t } }  } $$  
  1. 左侧${ \bar { { x }_{ t } }  }\left( t \right) $ : t次选择后, 每个臂的平均收益. 可见, 平均收益高的臂, 其评分也可能高  
  2. 右侧$\sqrt { \frac { 2\ln { t }  }{ { T }_{ j,t } }  } $ : 为奖励项, ${ T }_{ j,t }$表示t次试验中, 选择臂$j$的次数, $t$为总试验次数. 可见, 被选择次数少的臂其奖励就大.

### 6.1.4 Epsilon贪婪算法
#### 一.  Epsilon贪婪算法思路
1. 每次从(0,1)中随机选择一个数作为$\epsilon$, 再以$1-\epsilon$的概率选择平均收益最大的那个臂  
2. 由上所示, $\epsilon$越接近0, 就越会倾向于选择平均收益大的臂

### 6.1.5 冷启动问题

#### 一. 可以选择汤普森采样解决推荐系统冷启动问题
1. 使用分类或topic表示每个用户兴趣, 通过多次试验, 表示出用户对每个topic感兴趣的概率  
2. 新来一个用户, 抽取每个topic的随机数, 选择随机数最大的topic推荐给用户. 如果用户做出了反应, 该topic的$\alpha$值加1, 未作出反应, 该topic的$\beta$值加1